In [1]:
import pandas as pd


In [2]:
# Read list exported from TradingView
# Support comma or line delimited rows.

from io import StringIO

s = ','.join([x.rstrip().rstrip(',') for x in open('RUSSELL2000a.txt')] +
             [x.rstrip().rstrip(',') for x in open('RUSSELL2000b.txt')])
df1 = pd.read_csv(StringIO(s), sep=':', lineterminator=',', names=['Exchange', 'Ticker'])
# df1['Exchange'].value_counts()


In [3]:
# Read list downloaded from BlackRock iShares.
# https://www.ishares.com/us/products/239710/ishares-russell-2000-etf

fn = 'IWM_holdings.csv'

# Read data skipping preamble
df2 = pd.read_csv(fn, skiprows=9)

# Drop non-Equity, blank, unlisted rows
df2 = df2[(df2['Asset Class'] == 'Equity') &
          (df2['Ticker'] != '-') &
          (~df2['Ticker'].str.contains(r' |\d')) &
          (df2['Exchange'] != 'NO MARKET (E.G. UNLISTED)')]


In [4]:
# Update tickers
df2.loc[(df2['Ticker'] == 'LGFA') & (df2['Name'] == 'LIONS GATE ENTERTAINMENT VOTING CO'), 'Ticker'] = 'LGF.A'
df2.loc[(df2['Ticker'] == 'LGFB') & (df2['Name'] == 'LIONS GATE ENTERTAINMENT NON VOTIN'), 'Ticker'] = 'LGF.B'
df2.loc[(df2['Ticker'] == 'MOGA') & (df2['Name'] == 'MOOG INC CLASS A'), 'Ticker'] = 'MOG.A'
df2.loc[(df2['Ticker'] == 'CRDA') & (df2['Name'] == 'CRAWFORD CLASS A'), 'Ticker'] = 'CRD.A'
df2.loc[(df2['Ticker'] == 'GEFB') & (df2['Name'] == 'GREIF INC CLASS B'), 'Ticker'] = 'GEF.B'

# Drop delisted
df2.drop(df2[(df2['Ticker'] == 'PDLI') & (df2['Name'] == 'PDL BIOPHARMA INC')].index, inplace=True)
df2.drop(df2[(df2['Ticker'] == 'METCV') & (df2['Name'] == 'RAMACO RESOURCES INC CLASS A')].index, inplace=True)  # But METC is missing
df2.drop(df2[(df2['Ticker'] == 'GTXI') & (df2['Name'] == 'GTXI INC - CVR')].index, inplace=True)

# df2[(df2['Ticker'] == 'GTXI')]


In [5]:
lst = []

for index, row in df2.iterrows():
    exchange0 = row['Exchange']
    ticker = row['Ticker']

    d = {'NASDAQ': 'NASDAQ',
         'New York Stock Exchange Inc.': 'NYSE',
         'Nyse Mkt Llc': 'AMEX',
         'Non-Nms Quotation Service (Nnqs)': 'OTC'}
    try:
        exchange1 = d[exchange0]
    except KeyError as e:
        print(f'Unknown exchange {e} returned for ticker {ticker}, starting exchange is {exchange0}')
    else:
        lst += [[ticker, exchange0, exchange1]]

df3 = pd.DataFrame(lst, columns=['Ticker', 'OldExchange', 'NewExchange']).sort_values(by='Ticker')


In [6]:
removed_tickers = df1[~df1['Ticker'].isin(df3['Ticker'])]['Ticker']
added_tickers = df3[~df3['Ticker'].isin(df1['Ticker'])]['Ticker']


In [7]:
removed_tickers


Series([], Name: Ticker, dtype: object)

In [8]:
added_tickers


Series([], Name: Ticker, dtype: object)

In [9]:
df4 = (df3['NewExchange'] + ':' + df3['Ticker']).reset_index(drop=True)

(open('RUSSELL2000a_new.txt', 'w').write(',\n'.join(df4.iloc[:1000].tolist())),
 open('RUSSELL2000b_new.txt', 'w').write(',\n'.join(df4.iloc[1000:].tolist())))


(11851, 11260)